# Fine Tuning BERT 
fine-tuning of the model BERT

### Set the right GPU to use

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


### Import Libraries
Pandas for read the dataset and che transformers library to laod the model and the tokenizer

In [2]:
import transformers
import torch
import numpy as np
import random
import pandas as pd
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import ast
from sklearn.model_selection import train_test_split
import datasets
from datasets import Dataset, DatasetDict

In [3]:
from transformers import Trainer,TrainingArguments

2024-06-16 09:46:43.640969: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Read the dataset

In [ ]:
dataRew=pd.read_json("../Dataset/IMDB_reviews.json",lines=True)

In [ ]:
dataRew

drop useless coulumns

In [ ]:
dataRew.drop(columns=["movie_id","rating","review_date","user_id","review_summary"],inplace=True)

In [ ]:
dataRew

### Map true e false, in 0 e 1


In [ ]:
dataRew['is_spoiler'] = dataRew['is_spoiler'].map({True: 1, False: 0})

In [ ]:
dataRew = dataRew.rename(columns={'is_spoiler': 'label'})

### Split Dataset
We split the dataset using the stratify attribute to ensure that the divided sets are balanced.

### Train & Test

In [ ]:
train, test,= train_test_split(dataRew, test_size=0.2, stratify=dataRew['label'],random_state=42)

In [ ]:
train['label'].value_counts()

In [12]:
test['label'].value_counts()

label
0    84598
1    30185
Name: count, dtype: int64

### Divide in Valuation e Training

In [13]:
train, val,= train_test_split(train, test_size=0.2, stratify=train['label'],random_state=42)

In [14]:
train['label'].value_counts()

label
0    270713
1     96591
Name: count, dtype: int64

In [15]:
val['label'].value_counts()

label
0    67678
1    24148
Name: count, dtype: int64

In [16]:
test['label'].value_counts()

label
0    84598
1    30185
Name: count, dtype: int64

In [17]:
test

,label,review_text
322757,1,After seeing Sixth sense and Unbreakable i had...
280946,0,I rented this movie from my local video shop y...
269020,0,I was lucky enough to win tickets to an advanc...
342401,0,"Despite some awful critics, I have to say that..."
420689,0,"Found it a fascinating film as it went along, ..."
...,...,...
339747,0,Amazing. Absolutely astounding. Tom Cruise i...
11746,0,This movie defines excellence in the field of ...
45415,0,A gripping tale of murder and one of the best ...
321264,0,Having seen this movie twice over the weekend ...


In [18]:
train

,label,review_text
82889,0,Welcome to my Punner Island where all your pun...
198457,1,The movie is not bad. That's what I gotta say ...
330749,0,"An intense, depressing movie. It sticks pretty..."
57902,0,"If you are just an average movie fan, then you..."
71005,0,Their whole life is a solid game from start to...
...,...,...
545532,0,I absolutely hated Dumb & Dumber and only got ...
57859,0,I sat down through 2 hours of pure boredom. I ...
101697,0,The Wizard of Oz is an American classic undoub...
328531,1,I didn't like this movie at all. The plot was ...


### Change the  Dataset

In [19]:
Train = Dataset.from_pandas(train)
Eval = Dataset.from_pandas(val)
Test = Dataset.from_pandas(test)

In [20]:
Train=Train.remove_columns("__index_level_0__")
Eval=Eval.remove_columns("__index_level_0__")
Test=Test.remove_columns("__index_level_0__")

In [21]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

/home/f.caprari/prova/Group10venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Tokenization
Tokenization done with a maximum of 256 characters, specifying padding and truncation.

Try different max_lenght, try to increase the lenght, from 128 to 256

In [22]:
def encodeBig(text):
    return tokenizer(text['review_text'], padding="max_length", truncation=True, max_length=256)

In [23]:
Train=Train.map(encodeBig,batched=True)


Map:   0%|          | 0/367304 [00:00<?, ? examples/s]

In [24]:
Eval=Eval.map(encodeBig,batched=True)

Map:   0%|          | 0/91826 [00:00<?, ? examples/s]

In [25]:
Test=Test.map(encodeBig,batched=True)

Map:   0%|          | 0/114783 [00:00<?, ? examples/s]

In [26]:
from transformers import TrainingArguments, Trainer

#### Parameters of the models

In [27]:
BATCH_SIZE = 16
WEIGHT_DECAY=0.01
LR = 2e-5
EPOCHS = 5


In [28]:
torch.cuda.current_device()

0

In [29]:
#model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

In [30]:
model=AutoModelForSequenceClassification.from_pretrained("/opt/models/bert-base-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /opt/models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:


training_args = TrainingArguments(
output_dir="test_dir",
learning_rate=LR,
weight_decay=WEIGHT_DECAY,
num_train_epochs=EPOCHS,
evaluation_strategy="epoch",
per_device_train_batch_size=BATCH_SIZE,
per_device_eval_batch_size=BATCH_SIZE,
save_strategy='no',
fp16=True
)


### Function usefull to compute the metrics
We tried two different approaches: one using the **weighted metric**, which calculates the result of the metrics considering the number of samples, and the **binary metric**, which calculates the result only for the positive class.

In [32]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metricsweighted(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
    'accuracy': acc,
    'f1': f1,
    'precision': precision,
    'recall': recall
    }

In [33]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metricsbinary(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
    'accuracy': acc,
    'f1': f1,
    'precision': precision,
    'recall': recall
    }

In [34]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [35]:
#model=model.to(device)

In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train,
    eval_dataset=Eval,
    compute_metrics=compute_metricsbinary,
)

In [37]:
trainer.args.device

device(type='cuda', index=0)

In [38]:
history=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.479600,0.465602,0.784996,0.393885,0.761424,0.265653
2,0.436700,0.472862,0.788056,0.478203,0.678175,0.369306
3,0.395600,0.493134,0.774225,0.528261,0.586263,0.480702
4,0.332600,0.538723,0.775129,0.504998,0.599590,0.436185
5,0.262900,0.657036,0.769390,0.509270,0.578194,0.455027


In [39]:
history.metrics

{'train_runtime': 10811.1854,
 'train_samples_per_second': 169.872,
 'train_steps_per_second': 10.617,
 'total_flos': 2.416043576948736e+17,
 'train_loss': 0.3854855541693353,
 'epoch': 5.0}

In [40]:
log_history=trainer.state.log_history

In [42]:
with open("../Dataset/outputBert.txt", "a") as f:
    for log in log_history:
         if 'eval_loss' in log:
            print(f"Epoch: {log.get('epoch')}, Eval Loss: {log['eval_loss']}, Accuracy: {log['eval_accuracy']}, F1: {log['eval_f1']}, Precision: {log['eval_precision']}, Recall: {log['eval_recall']}",file=f)

In [43]:
evaluationRes=trainer.evaluate()

In [44]:
evaluationRes

{'eval_loss': 0.6570364236831665,
 'eval_accuracy': 0.7693899331344064,
 'eval_f1': 0.5092695587690026,
 'eval_precision': 0.5781940644074932,
 'eval_recall': 0.4550273314560212,
 'eval_runtime': 167.0954,
 'eval_samples_per_second': 549.542,
 'eval_steps_per_second': 34.352,
 'epoch': 5.0}

In [45]:
log_history=trainer.evaluate(Test)

In [46]:
with open("../Dataset/outputBert.txt", "a") as f:
    print("Result on Test",file=f)
    print(f"Eval Loss: {log_history['eval_loss']}, Accuracy: {log_history['eval_accuracy']}, F1: {log_history['eval_f1']}, Precision: {log_history['eval_precision']}, Recall: {log_history['eval_recall']}",file=f)